# Создание эмбеддингов традиционным и новым методами
В этом ноутбуке датасет https://huggingface.co/datasets/ai-forever/ria-news-retrieval преобразован в эмбеддинги

In [1]:
from chunked_pooling import chunked_pooling, chunk_by_sentences
from transformers import AutoModel
from transformers import AutoTokenizer
import pandas as pd
import torch
import numpy as np
import os

In [2]:
basePath = os.path.abspath('')
queries = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\queries.jsonl", lines=True)
corpus = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\corpus.jsonl", lines=True)
test = pd.read_json(basePath + "\\ai-forever-ria-news-retrieval\\test.jsonl", lines=True)
del corpus['title']
del test['score']

### Единственное проблемное место датасета, но это было исправлено в будущем

In [5]:
corpus[corpus['text'] == ""]

,_id,text
3840,3840,


In [3]:
def do_ch_emb(calc_chunk):
    device = torch.device("cuda")
    # load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True, device_map = 'cuda')
    model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True, device_map = 'cuda')
    result = pd.DataFrame({
        "chunk" : [],
        "trad_chunk_embedding" : [],
        "new_chunk_embedding" : [],
        "doc_id" : [],
    })
    result.astype('object')
    last_idx = 0
    for doc_id, doc in calc_chunk.iterrows():
        doc_chunks, doc_span_annotations = chunk_by_sentences(doc['text'], tokenizer)
        doc_trad_chunk_embeddings = model.encode(doc_chunks)
        doc_inputs = tokenizer(doc['text'], return_tensors='pt')
        doc_model_output = model(**(doc_inputs.to(device)))
        doc_new_chunk_embeddings = chunked_pooling(doc_model_output, [doc_span_annotations])[0]
        for i, (chunk, trad_chunk_embedding, new_chunk_embedding) in enumerate(zip(doc_chunks, doc_trad_chunk_embeddings, doc_new_chunk_embeddings)):
            result.loc[last_idx + i] = np.array([chunk, trad_chunk_embedding, new_chunk_embedding, doc['_id']], dtype=object)
        last_idx += len(doc_chunks)
    return result

### Для ускорения рассчётов данные были разбиты на 22 куска для парааллельной обработки на разных машинах

In [5]:
chunked_corpus  = np.array_split(corpus, 22)

C:\Users\gorku\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
%%time
chunks12 = do_ch_emb(chunked_corpus[12])
chunks12.shape

flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn is not installed. Using PyTorch native attention implementation.
flash_attn i

CPU times: total: 2h 54min 26s
Wall time: 4h 53min 9s


(252887, 4)

In [6]:
chunks12.to_pickle(basePath + "\\ai-forever-ria-news-retrieval\\chunks_embedded_12.pkl")

### Эмбеддинг запросов

In [ ]:
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True, device_map = 'cuda')

In [ ]:
%%time
queries['embedding'] = queries.apply(lambda q: model.encode(q['text']), axis=1)

In [ ]:
queries.head()

In [ ]:
queries.to_pickle(basePath + "\\ai-forever-ria-news-retrieval\\queries_embedded.pkl")